In [23]:
%pip install --upgrade openai
#%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable




[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\koichino\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### アシスタントを作成する。

In [1]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("SWC_AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("SWC_AZURE_OPENAI_ENDPOINT")
    )

# Upload a file with an "assistants" purpose
# csv = client.files.create(
#     file=open(r"data\Population in Japan.csv", "rb"),
#     purpose='assistants'
# )
csv_id="assistant-4q4unqEhyqXPDuFuKYFkVDRq"

# fontfile = client.files.create(
#     file=open(r"data\Noto_Sans_JP.zip", "rb"),
#     purpose='assistants'
# )
fontfile_id="assistant-J2RNrMbBKIWGQ914fyY47AUx"

# Create an assistant
assistant = client.beta.assistants.create(
    name="Diabetes Assistant",
    instructions=f"あなたは医療アシスタントです。" 
    f"特に糖尿病に関する分析を得意としてます。"
    f"分析する際には以下の手順でグラフ化するようにしてください。"
    f"1. アップロードされたフォントファイルが含まれたZIPを解凍する。"
    f"2. 解凍されたフォルダからフォントファイル(拡張子ttf)を読み込む。"
    f"3. グラフ表示の際に、読み込んだフォントで出力する。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-35-turbo-16k" #You must replace this value with the deployment name for your model.
    ,tool_resources={
        "code_interpreter": {
        "file_ids": [csv_id,fontfile_id]
        }
    }
    #,file_ids=[csv_id, fontfile_id]
)


In [2]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "糖尿病CSVファイルの概要を教えて。各列毎の説明もつけてください。",

      #"file_ids":[csv_id, fontfile_id] # file id will look like: "assistant-R9uhPxvRKGH3m0x5zBOhMjd2" 
    },
  ],
  tool_resources ={
        "code_interpreter": {
        "file_ids": [csv_id,fontfile_id]
          }
      }
)

TypeError: Threads.create() got an unexpected keyword argument 'tool_resourcesa'

In [30]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_PjfzbpLH2fDttYGLkvcfex2T",
  "created_at": 1721888510,
  "description": null,
  "instructions": "あなたは医療アシスタントです。特に糖尿病に関する分析を得意としてます。分析する際には以下の手順でグラフ化するようにしてください。1. アップロードされたフォントファイルが含まれたZIPを解凍する。2. 解凍されたフォルダからフォントファイル(拡張子ttf)を読み込む。3. グラフ表示の際に、読み込んだフォントで出力する。",
  "metadata": {},
  "model": "gpt-35-turbo-16k",
  "name": "Diabetes Assistant",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": [
        "assistant-4q4unqEhyqXPDuFuKYFkVDRq",
        "assistant-J2RNrMbBKIWGQ914fyY47AUx"
      ]
    },
    "file_search": null
  },
  "top_p": 1.0
}


In [32]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))


{
  "data": [
    {
      "id": "msg_wVCxPYlPcW1ite4sOnYHrrDm",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "糖尿病CSVファイルの概要を教えて。各列毎の説明もつけてください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1721889974,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_nhCe2SUQ4Xry6ag6JhHfzUvH"
    }
  ],
  "object": "list",
  "first_id": "msg_wVCxPYlPcW1ite4sOnYHrrDm",
  "last_id": "msg_wVCxPYlPcW1ite4sOnYHrrDm",
  "has_more": false
}


In [56]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [57]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

KeyboardInterrupt: 

In [58]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_56bBjeFnsnPy6oaSTGQdOeiT",
      "assistant_id": "asst_PjfzbpLH2fDttYGLkvcfex2T",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "申し訳ありませんが、エラーの原因を特定できませんでした。エラーを修正するために必要な手順を特定し、グラフを作成しますので、しばらくお待ちください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1721898547,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_C6AeY8LrB2bovjYFeAhBZ9YH",
      "status": null,
      "thread_id": "thread_nhCe2SUQ4Xry6ag6JhHfzUvH"
    },
    {
      "id": "msg_W4S7Pi0m3BqNlBAtTcx1QMnM",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "お願いします。"
          },
          "type": "text"
      

In [39]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="糖尿病疾患がある人に影響している要素を分析したいです。グラフを作成して説明してください。グラフのタイトルは、「糖尿病に影響する要素について」としてください。"
)

In [40]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [55]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="お願いします。"
)

In [52]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)